In [1]:
import numpy as np
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import os
from img2vec_pytorch import Img2Vec
from PIL import Image
import xml.etree.ElementTree as ET

In [14]:
elastic_client = Elasticsearch(hosts=['http://localhost:9200'],
                               basic_auth=('elastic', 'M6fSr6FCABQhGwqR8HPf'))
model = Img2Vec(True)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [3]:
def generate_embedding(segment):
    #img = Image.open(filename).convert("RGB")
    vec = model.get_vec(segment)
    return vec.tolist()

In [4]:
def build_doc(no:str, name:str, classes, vector):
  document={ 
    "no": no,
    "name": name,
    "class_label": classes, 
    "embedding": vector
  }   
  return document

In [5]:
def get_files(path:str, extension:str):
    file_list = [f for f in os.listdir(path) if f.endswith(extension)]
    return file_list

In [6]:
#{[
#    {class: 'd10', segment_values: [100,100,100,100]},
#    {class: 'd40', segment_values: [100,100,100,100]}
#]}

def get_classes_segments(path:str, file_name:str):
    segments = []
    class_ = {}
    values = []
    tree = ET.parse(path+file_name)
    root = tree.getroot()
    for child in root:
        if (child.tag == 'object'):
            for new_child in child:
                if (new_child.tag == 'name'):
                    class_ = new_child.text
                if (new_child.tag == 'bndbox'):
                    for bndbox in new_child:
                        values.append(int(bndbox.text)) 
                    segments.append({'class': class_, 'segment_values': values})
                    values = []            
    return segments

In [11]:
dataset_path = "./datasets/road/train/"
index_name = "road_image_segment"
#segment = get_classes_segments(dataset_path, "4_png_jpg.rf.b3f95d13379e2606209b18e200655512.xml")
#print(segment)

In [12]:
id = 0
file_id = 0
actions = []
file_list = get_files(dataset_path, ".jpg")
actions = []
for file in file_list: 
    file_id += 1
    image_file = dataset_path+file
    xml_file = file[0:len(file)-4]+".xml"
    classes_segments = get_classes_segments(dataset_path, xml_file)
    image = Image.open(image_file).convert("RGB")
    print("Image: ", file_id, " -> ", image_file)
    for class_segment in classes_segments:
      id += 1
      class_ = class_segment.get("class")
      values = class_segment.get("segment_values") 
      xmin = values[0]
      xmax = values[1]
      ymin = values[2]
      ymax = values[3]
      segment = image.crop((xmin,ymin,xmax,ymax))
      vector = generate_embedding(segment)
      actions.append({"_index":index_name.lower(), 
                    "_source": build_doc(id, file, class_, vector)})
      if ((id % 50 ) == 0):
        bulk(elastic_client, actions)
        actions.clear()
        print("{} segments were indexed!!!".format(id))

if len(actions) != 0:
  bulk(elastic_client, actions)
  actions.clear()
  print("{} segments were indexed!!!".format(id))
print("Indexing was finished!!!")


Image:  1  ->  ./datasets/road/train/26_png_jpg.rf.9a5ddca4ac398c6ddb37f8c822768bb6.jpg
Image:  2  ->  ./datasets/road/train/31_png_jpg.rf.19a17108040fa4187df1e05562b792bb.jpg
Image:  3  ->  ./datasets/road/train/44_png_jpg.rf.ea9284c7e6eea14e7d3712179ea3eba6.jpg
Image:  4  ->  ./datasets/road/train/China_Drone_000001_jpg.rf.5c775cf8c3dfc3569cba5ad4a766757c.jpg
Image:  5  ->  ./datasets/road/train/China_Drone_000051_jpg.rf.f56d48eb38cda9bc6bacc6d3ab677b52.jpg
Image:  6  ->  ./datasets/road/train/China_Drone_000042_jpg.rf.10397ad3862c9094e77fbde8f9b9245b.jpg
Image:  7  ->  ./datasets/road/train/43_png_jpg.rf.0580fc0caf391701a3e72532228c2145.jpg
Image:  8  ->  ./datasets/road/train/China_Drone_000054_jpg.rf.30c8d4312f9b2405ec3a4e900ccfdab9.jpg
Image:  9  ->  ./datasets/road/train/China_Drone_000001_jpg.rf.a6c4ab36b194b46f0832e89785040e35.jpg
Image:  10  ->  ./datasets/road/train/59_png_jpg.rf.4b853b3af78d9ad5dd412fc9ede575fc.jpg
Image:  11  ->  ./datasets/road/train/China_Drone_000022_jp

POST http://localhost:9200/_bulk [status:N/A request:0.001s]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/opt/conda/lib/python3.11/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/elasticsearch/connection/http_urllib3.py", line 172, in perform_request
    response = self.pool.urlopen(method, url, body, retries=Retry(False), headers=request_headers, **kw)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Image:  17  ->  ./datasets/road/train/China_Drone_000023_jpg.rf.aa3c7990952153e05a5bf8acddb1873b.jpg


POST http://localhost:9200/_bulk [status:N/A request:0.001s]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/opt/conda/lib/python3.11/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/elasticsearch/connection/http_urllib3.py", line 172, in perform_request
    response = self.pool.urlopen(method, url, body, retries=Retry(False), headers=request_headers, **kw)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7f78bcbda3d0>: Failed to establish a new connection: [Errno 111] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7f78bcbda3d0>: Failed to establish a new connection: [Errno 111] Connection refused)